In [134]:
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data
import yfinance as yf
%matplotlib inline

# For defining dates
import datetime as dt

# Read from sql
import sqlite3

# Default Values

In [135]:
# Define path to files
PATH = "/Users/jwiegand/Dev/jrwiegand/data/files/"

# Start date defaults
S_YEAR = 2021
S_MONTH = 6
S_DAY = 1
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# Start date defaults
E_YEAR = 2024
E_MONTH = 6
E_DAY = 1
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)


# Manually Download the Screener CSV from Nasdaq
# https://www.nasdaq.com/market-activity/stocks/screener
CSV_DATA_FILE_NAME = "nasdaq_screener_1717356628553.csv"

# Stocks that were not downloaded
stocks_not_downloaded = []

# Import Data from CSV to SQL

In [136]:
def import_csv_to_sqlite(csv_file, db_file, table_name):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Connect to the SQLite database (creates it if it doesn't exist)
    connection = sqlite3.connect(db_file)

    df.to_sql("stocks", connection, if_exists="replace", index=True)

    connection.close()
    print(f"Data from {csv_file} successfully imported into {db_file} in table {table_name}")

# Returns a Named Column Data from a CSV

In [137]:
def get_column_from_csv(file, col_name):
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(file)
        df[col_name] = df[col_name].astype(str)
        df[col_name] = df[col_name].apply(str)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df[col_name].astype("string")

# Function that Saves Stock Data to CSV

In [138]:
# Function that gets a dataframe by providing a ticker and starting date
def save_to_csv_from_yahoo(folder, stock_symbol, start, end):
    try:
        print("Get Data for: ", stock_symbol)

        yf.pdr_override()

        # Reads data into a dataframe
        df = web.DataReader(stock_symbol, start, end)["Adj Close"]

        # Save data to a CSV file
        df.to_csv(folder + stock_symbol + ".csv")
    except Exception as ex:
        print(ex)
        stocks_not_downloaded.append(stock_symbol)
        print("Could Not Get Data for :", stock_symbol)

# Test Receiving Stock Tickers

In [139]:
import_csv_to_sqlite(PATH + CSV_DATA_FILE_NAME, PATH + "stocks.db", "stocks")

Data from /Users/jwiegand/Dev/jrwiegand/data/files/nasdaq_screener_1717356628553.csv successfully imported into /Users/jwiegand/Dev/jrwiegand/data/files/stocks.db in table stocks


# Get 5 Years of Data for all the Stocks

In [140]:
connection = sqlite3.connect(PATH + "stocks.db")
cursor = connection.cursor()
cursor.execute("SELECT Symbol FROM stocks")
rows = cursor.fetchall()

for row in rows:
  symbol = row[0]
  save_to_csv_from_yahoo(PATH + "stocks/", symbol, S_DATE_DATETIME, E_DATE_DATETIME)

print("Finished")
print("Stocks Not Downloaded: ", stocks_not_downloaded)

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['A']: %ticker%ttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AA']: AttributeError("'str' object has no attribute 'get'")


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AACG']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AACI']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AACIU']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AACIW']: AttributeError("'str' object has no attribute 'get'")


Get Data for:  A
Get Data for:  AA
Get Data for:  AACG
Get Data for:  AACI
Get Data for:  AACIU
Get Data for:  AACIW


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AACT']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AADI']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAGR']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAL']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAMC']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAME']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed down

Get Data for:  AACT
Get Data for:  AADI
Get Data for:  AAGR
Get Data for:  AAL
Get Data for:  AAMC
Get Data for:  AAME
Get Data for:  AAN
Get Data for:  AAOI
Get Data for:  AAON
Get Data for:  AAP
Get Data for:  AAPL
Get Data for:  AAT


...

Get Data for:  ZKH
Get Data for:  ZKIN
Get Data for:  ZLAB
Get Data for:  ZLS
Get Data for:  ZLSWU
Get Data for:  ZLSWW
Get Data for:  ZM
Get Data for:  ZNTL
Get Data for:  ZOM
Get Data for:  ZONE
Get Data for:  ZOOZ
Get Data for:  ZOOZW
Get Data for:  ZPTA
Get Data for:  ZPTAW


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZPTAW']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZS']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZTEK']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZTO']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZTR']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZTS']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed downloa

Get Data for:  ZS
Get Data for:  ZTEK
Get Data for:  ZTO
Get Data for:  ZTR
Get Data for:  ZTS
Get Data for:  ZUMZ
Get Data for:  ZUO
Get Data for:  ZURA
Get Data for:  ZURAW
Get Data for:  ZVIA
Get Data for:  ZVRA
Get Data for:  ZVSA
Get Data for:  ZWS
Get Data for:  ZYME


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZYME']: AttributeError("'str' object has no attribute 'get'")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZYXI']: AttributeError("'str' object has no attribute 'get'")


Get Data for:  ZYXI
Finished
Stocks Not Downloaded:  ['AKO/A', 'AKO/B', 'BF/A', 'BF/B', 'BIO/B', 'BRK/A', 'BRK/B', 'CRD/A', 'CRD/B', 'HEI/A', 'HVT/A', None, 'WSO/B']
